In [ ]:
using Distributions, Random, Plots, CurveFit, LinearAlgebra, Richardson
Random.seed!(123)

In [ ]:
global const diffusivity = 1.00::Float64

global const endtime = 1.00::Float64
global const L = 1.00::Float64

global const dt = 1e-4::Float64
global const timesteps = Int64(div(endtime,dt)+1)::Int64

global const spacesteps = 51::Int64

global const u0 = 1::Int64
global const u0_tilda = (u0/spacesteps)::Float64

global const cfl = (diffusivity*(spacesteps*spacesteps)*dt)::Float64


### Diffusion equation 
$$\frac{\partial u}{\partial t} = \frac{\partial}{\partial x} \left( D\frac{\partial u}{\partial x}\right)$$

In [ ]:
function Crank_Nicolson_step(A, B, u_inint)
    return A\(B*u_inint)
end

function CN_absorbing_BC(u_init, alpha)
    #A * U_n+1 = B * U_n
    n_steps = spacesteps-2

    A = Tridiagonal(fill(-alpha/2, n_steps-1),
                    fill(1+alpha, n_steps),
                    fill(-alpha/2, n_steps-1))

    B = Tridiagonal(fill(alpha/2, n_steps-1),
                    fill(1-alpha, n_steps),
                    fill(alpha/2, n_steps-1))


    res = Array{Array{Float64, 1}, 1}(undef, timesteps)
    res[1] = u_init
    for i in 2:timesteps
        res[i] = reduce(vcat, [0, Crank_Nicolson_step(A,B,res[i-1][2:end-1]),0])
    end
    return res
end 

function CN_reflecting_BC(u_init, alpha)
    #A * U_n+1 = B * U_n
    n_steps = size(u_init,1)

    A = Tridiagonal(reduce(vcat, [fill(-alpha/2, n_steps-2), -alpha]),
                        fill(1+alpha, n_steps),
                        reduce(vcat, [-alpha, fill(-alpha/2, n_steps-2)]))
    
    B = Tridiagonal(reduce(vcat, [fill(alpha/2, n_steps-2), alpha]),
                        fill(1-alpha, n_steps),
                        reduce(vcat, [alpha, fill(alpha/2, n_steps-2)]))

    res = Array{Array{Float64, 1}, 1}(undef, timesteps)
    res[1] = u_init
    for i in 2:timesteps
        res[i] = Crank_Nicolson_step(A,B,res[i-1])
    end
    return res
end



In [ ]:
function unbounded(x0, x, t)
    return u0_tilda/sqrt(4 * pi * diffusivity * t) * exp(-(x-x0)^2/(4* diffusivity * t))
end

In [ ]:
x = reduce(vcat, [0, cumsum(fill(L/(spacesteps-1), spacesteps-1))])
t = LinRange(0,endtime, timesteps)

u_init = zeros(spacesteps)
u_init[div(spacesteps, 2)+1] = u0

u_res_absorbing = CN_absorbing_BC(u_init, cfl)

u_res_reflecting = CN_reflecting_BC(u_init, cfl)

u_anal_unbounded = unbounded.(0.5, x', t[2:end])


In [ ]:
plot(scatter(x,u_res_absorbing[100], label = t[100]))
plot!(scatter!(x,u_res_absorbing[500], label = t[500]))
plot!(scatter!(x,u_res_absorbing[750], label = t[750]))
plot!(scatter!(x,u_res_absorbing[10000], label = t[10000]))

plot!(x, u_anal_unbounded[100, :], label = t[100])
plot!(x, u_anal_unbounded[500, :], label = t[500])
plot!(x, u_anal_unbounded[750, :], label = t[750])
plot!(x, u_anal_unbounded[end, :], label = t[end])

In [ ]:
print(sum(u_res_reflecting[100]))
plot(scatter(x,u_res_reflecting[100], label = t[100]))
plot!(scatter!(x,u_res_reflecting[500], label = t[500]))
plot!(scatter!(x,u_res_reflecting[750], label = t[750]))
plot!(scatter!(x,u_res_reflecting[10000], label = t[10000]))

plot!(x, u_anal_unbounded[100, :], label = t[100])
plot!(x, u_anal_unbounded[500, :], label = t[500])
plot!(x, u_anal_unbounded[750, :], label = t[750])
plot!(x, u_anal_unbounded[end, :], label = t[end])
